In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches

In [2]:
df = pd.read_csv(r'../../results/computed-fuel-rates-oct2021-sep2022.csv', delimiter=',', skiprows=0, low_memory=False)

In [3]:
df.columns

Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'ServiceDateTime', 'DateKey', 'CalendarDate', 'Year',
       'Month', 'OpKey', 'Operator', 'RtKey', 'RtKey2', 'Route', 'RouteName',
       'TripKey', 'Trip', 'StopKey', 'Stop', 'StopName', 'Vehicle',
       'VehicleType', 'SeatedCap', 'TotalCap', 'Lat', 'Lon', 'Boards',
       'Alights', 'Onboard', 'Bike', 'Daytype', 'Hour', 'Minute', 'Second',
       'Date', 'Powertrain', 'VehicleModel', 'Vehicle_mass',
       'time_delta_in_seconds', 'dist', 'Speed', 'Acceleration', 'elevation',
       'grade', 'Energy'],
      dtype='object')

In [18]:
grouped = df.groupby(['Route', 'Month'])['Vehicle'].unique()
grouped_df = grouped.reset_index()

In [19]:
grouped_df

,Route,Month,Vehicle
0,(X) 34,4,"[3306, 3307, 3308, 3309, 3310, 3311, 3312, 3313]"
1,(X) 38,4,"[3212, 3222, 3225, 3226, 3228, 3301, 3302, 330..."
2,(X) 39E,4,"[1205, 5574, 7660, 7802, 7811, 7812, 7813, 781..."
3,(X) 801,4,"[1321, 1322, 1415, 1420, 1421, 1557, 1563, 156..."
4,(X) 802,4,"[1321, 1411, 1413, 1415, 1416, 1552, 1557, 156..."
...,...,...,...
567,X92,8,"[1320, 1321, 1322, 1323, 1555, 1567, 1571, 157..."
568,X92,9,"[1320, 1321, 1323, 1563, 1564, 1565, 1566, 156..."
569,X92,10,"[1510, 1515, 1552, 1553, 1554, 1555, 1556, 155..."
570,X92,11,"[1321, 1416, 1513, 1515, 1553, 1554, 1555, 155..."


In [11]:
grouped = df.groupby(['Route'])['Vehicle'].unique()
grouped_df = grouped.reset_index()

In [21]:
unique_vehicles = {}
common_vehicles = {}

# Iterate over each route
for _, group in grouped_df.groupby(['Route']):
    route = _[0]
    vehicles_by_month = {}

    # Iterate over each month within the route
    for _, row in group.iterrows():
        month = row['Month']
        vehicles = set(row['Vehicle'])

        if month not in vehicles_by_month:
            vehicles_by_month[month] = vehicles
        else:
            vehicles_by_month[month] = vehicles_by_month[month].union(vehicles)

    # Compare the vehicles for each pair of months
    for i, month1 in enumerate(vehicles_by_month.keys()):
        for j, month2 in enumerate(vehicles_by_month.keys()):
            if j > i:
                vehicles1 = vehicles_by_month[month1]
                vehicles2 = vehicles_by_month[month2]

                common = vehicles1.intersection(vehicles2)
                unique1 = vehicles1.difference(vehicles2)
                unique2 = vehicles2.difference(vehicles1)

                if route not in common_vehicles:
                    common_vehicles[route] = {}
                common_vehicles[route][f"{month1} vs {month2}"] = common

                if route not in unique_vehicles:
                    unique_vehicles[route] = {}
                unique_vehicles[route][f"{month1} vs {month2}"] = (unique1, unique2)

# Print the results
for route in common_vehicles:
    print(f"Route: {route}")
    print("Common Vehicles:")
    for months, common in common_vehicles[route].items():
        print(f"{months}: {common}")
    print("Unique Vehicles:")
    for months, (unique1, unique2) in unique_vehicles[route].items():
        print(f"{months}: {unique1} (Unique in Month 1)")
        print(f"{months}: {unique2} (Unique in Month 2)")
    print()


Route: (
Common Vehicles:
10 vs 11: {1671, 1840, 1648, 1554, 1720}
10 vs 12: {1578, 1660}
11 vs 12: {1672, 1721, 1567, 1719}
Unique Vehicles:
10 vs 11: {1568, 1569, 1857, 1860, 1573, 1578, 1515, 1647, 1712, 1649, 1714, 1556, 1817, 1722, 1851, 1660} (Unique in Month 1)
10 vs 11: {1415, 1672, 1803, 1552, 1553, 1814, 1561, 1566, 1567, 1719, 1721, 1849, 1723, 1618, 1513, 1645, 1654, 1656, 1661} (Unique in Month 2)
10 vs 12: {1671, 1554, 1556, 1817, 1568, 1569, 1573, 1712, 1840, 1714, 1720, 1722, 1851, 1857, 1860, 1515, 1647, 1648, 1649} (Unique in Month 1)
10 vs 12: {1672, 1832, 1719, 1721, 1567} (Unique in Month 2)
11 vs 12: {1671, 1415, 1803, 1552, 1553, 1554, 1814, 1561, 1566, 1840, 1720, 1849, 1723, 1618, 1513, 1645, 1648, 1654, 1656, 1661} (Unique in Month 1)
11 vs 12: {1832, 1578, 1660} (Unique in Month 2)

Route: 3
Common Vehicles:
1 vs 2: {7811, 1205, 5574, 7814}
1 vs 3: {7801, 7811, 1205, 5574}
1 vs 4: {3306, 3307, 3308, 3309, 3311, 3312, 3314}
1 vs 5: {7814, 1205, 5574, 7551}
1 v

C:\Users\Mahsa\AppData\Local\Temp\ipykernel_3776\3200339558.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped_df.groupby(['Route']):


In [17]:
grouped_df

,Route,Vehicle
0,(X) 34,"[3306, 3307, 3308, 3309, 3310, 3311, 3312, 3313]"
1,(X) 38,"[3212, 3222, 3225, 3226, 3228, 3301, 3302, 330..."
2,(X) 39E,"[1205, 5574, 7660, 7802, 7811, 7812, 7813, 781..."
3,(X) 801,"[1321, 1322, 1415, 1420, 1421, 1557, 1563, 156..."
4,(X) 802,"[1321, 1411, 1413, 1415, 1416, 1552, 1557, 156..."
...,...,...
65,R41,"[1648, 7550, 7551, 7552, 7553, 7554, 7610, 766..."
66,R42,"[1648, 7550, 7551, 7552, 7553, 7554, 7610, 766..."
67,R44,"[1205, 5574, 7550, 7551, 7552, 7553, 7554, 761..."
68,X90,"[1320, 1321, 1322, 1323, 1415, 1420, 1421, 151..."


In [15]:
len(grouped_df.loc[grouped_df['Route'] == '(X) 34'].values.tolist())

1